# Test fibermap combination to avoid FITS errors

## Replicate `assemble_fibermap` script

```
assemble_fibermap -n NIGHT -e EXPID -o OUTFILE
```

### Set up options

In [2]:
import os
import sys
import argparse
from unittest.mock import patch
import glob
import warnings
import time
from pkg_resources import resource_filename

import yaml

import numpy as np
from astropy.table import Table, Column, join
from astropy.io import fits

from desitarget.targetmask import desi_mask

from desiutil.log import get_logger
from desiutil.depend import add_dependencies

from desispec.io.util import fitsheader, write_bintable, makepath, addkeys, parse_badamps
from desispec.io.meta import rawdata_root, findfile, faflavor2program
from desispec.io import iotime

from desispec.maskbits import fibermask
from desispec.io.fibermap import find_fiberassign_file, compare_fiberassign, assemble_fibermap

In [3]:
os.environ['DESI_LOGLEVEL'] = 'DEBUG'
os.environ['SPECPROD'] = 'everest'
if 'CSCRATCH' not in os.environ:
    os.environ['CSCRATCH'] = os.path.join(os.environ['HOME'], 'Documents', 'Data', 'scratch')
night = '20210922'
expid = '00101293'
outfile = os.path.join(os.environ['CSCRATCH'], f'fibermap-{expid}.fits')

with patch('sys.argv', ['assemble_fibermap', '-n', night, '-e', expid, '-o', outfile]) as foo: 
    parser = argparse.ArgumentParser(usage = "{prog} [options]")
    parser.add_argument("-n", "--night", type=int, required=True,
            help="input night")
    parser.add_argument("-e", "--expid", type=int, required=True,
            help="spectroscopic exposure ID")
    parser.add_argument("-o", "--outfile", type=str, required=True,
            help="output filename")
    parser.add_argument("-b","--badamps", type=str,
            help="comma separated list of {camera}{petal}{amp}"+\
                 ", i.e. [brz][0-9][ABCD]. Example: 'b7D,z8A'")
    parser.add_argument("--badfibers", type=str,
            help="filename with table of bad fibers (with at least FIBER and FIBERSTATUS columns)")
    parser.add_argument("--debug", action="store_true",
            help="enter ipython debug mode at end")
    parser.add_argument("--overwrite", action="store_true",
            help="overwrite pre-existing output file")
    parser.add_argument("--force", action="store_true",
            help="make fibermap even if missing input guide or coordinates files")
    parser.add_argument("--no-svn-override", action="store_true",
            help="Do not allow fiberassign SVN to override raw data")

    args = parser.parse_args()

print(args)


Namespace(badamps=None, badfibers=None, debug=False, expid=101293, force=False, night=20210922, no_svn_override=False, outfile='/global/cscratch1/sd/bweaver/fibermap-00101293.fits', overwrite=False)


### Run `assemble_fibermap()`

In [ ]:
# fibermap = assemble_fibermap(args.night, args.expid, badamps=args.badamps, force=args.force)
fibermap = assemble_fibermap(args.night, args.expid, badamps=args.badamps, badfibers_filename=args.badfibers, force=args.force, allow_svn_override=(not args.no_svn_override) )

### Write file

In [ ]:
tmpfile = args.outfile+'.tmp'
fibermap.write(tmpfile, overwrite=args.overwrite, format='fits')
os.rename(tmpfile, args.outfile)
# log.info(f'Wrote {args.outfile}')

In [ ]:
fibermap

In [ ]:
fibermap.meta

## Recreate `assemble_fibermap()`

### Examine raw file

In [ ]:
log = get_logger()

rawfile = findfile('raw', night, int(expid))
try:
    rawheader = fits.getheader(rawfile, 'SPEC', disable_image_compression=True)
except KeyError:
    rawheader = fits.getheader(rawfile, 'SPS', disable_image_compression=True)
rawfafile = fafile = find_fiberassign_file(night, int(expid))


In [4]:
rawfafile

'/Users/weaver/Documents/Data/desi/spectro/data/20210922/00101293/fiberassign-025535.fits.gz'

In [5]:
allow_svn_override=(not args.no_svn_override)
#- Look for override fiberassign file in svn
tileid = rawheader['TILEID']
if allow_svn_override and ('DESI_TARGET' in os.environ):
    targdir = os.getenv('DESI_TARGET')
    testfile = f'{targdir}/fiberassign/tiles/trunk/{tileid//1000:03d}/fiberassign-{tileid:06d}.fits'
    if os.path.exists(testfile+'.gz'):
        fafile = testfile+'.gz'
    elif os.path.exists(testfile):
        fafile = testfile

    if rawfafile != fafile:
        log.info(f'Overriding raw fiberassign file {rawfafile} with svn {fafile}')
    else:
        log.info(f'{testfile}[.gz] not found; sticking with raw data fiberassign file')


INFO:<ipython-input-5-61b637b7071f>:13:<module>: Overriding raw fiberassign file /Users/weaver/Documents/Data/desi/spectro/data/20210922/00101293/fiberassign-025535.fits.gz with svn /Users/weaver/Documents/Data/desi/target/fiberassign/tiles/trunk/025/fiberassign-025535.fits.gz


In [6]:
force=args.force
#- Find coordinates file in same directory
dirname, filename = os.path.split(rawfafile)
globfiles = glob.glob(dirname+'/coordinates-*.fits')
if len(globfiles) == 1:
    coordfile = globfiles[0]
elif len(globfiles) == 0:
    message = f'No coordinates*.fits file in fiberassign dir {dirname}'
    if force:
        log.error(message + '; continuing anyway')
        coordfile = None
    else:
        raise FileNotFoundError(message)

elif len(globfiles) > 1:
    raise RuntimeError(f'Multiple coordinates*.fits files in fiberassign dir {dirname}')


In [7]:
#- And guide file
dirname, filename = os.path.split(rawfafile)
globfiles = glob.glob(dirname+'/guide-????????.fits.fz')
if len(globfiles) == 0:
    #- try falling back to acquisition image
    globfiles = glob.glob(dirname+'/guide-????????-0000.fits.fz')

if len(globfiles) == 1:
    guidefile = globfiles[0]
elif len(globfiles) == 0:
    message = f'No guide-*.fits.fz file in fiberassign dir {dirname}'
    if force:
        log.error(message + '; continuing anyway')
        guidefile = None
    else:
        raise FileNotFoundError(message)

elif len(globfiles) > 1:
    raise RuntimeError(f'Multiple guide-*.fits.fz files in fiberassign dir {dirname}')


In [8]:
coordfile, guidefile

('/Users/weaver/Documents/Data/desi/spectro/data/20210922/00101293/coordinates-00101293.fits',
 '/Users/weaver/Documents/Data/desi/spectro/data/20210922/00101293/guide-00101293.fits.fz')

In [9]:
#- Read QA parameters to find max offset for POOR and BAD positioning
#- replicates desispec.exposure_qa.get_qa_params, but that has
#- circular import if loaded from here
param_filename = resource_filename('desispec', 'data/qa/qa-params.yaml')
with open(param_filename) as f:
    qa_params = yaml.safe_load(f)['exposure_qa']

poor_offset_um = qa_params['poor_fiber_offset_mm']*1000
bad_offset_um = qa_params['bad_fiber_offset_mm']*1000

#- Preflight announcements
log.info(f'Night {night} spectro expid {expid}')
log.info(f'Raw data file {rawfile}')
log.info(f'Fiberassign file {fafile}')
if fafile != rawfafile:
    log.info(f'Original raw fiberassign file {rawfafile}')
log.info(f'Platemaker coordinates file {coordfile}')
log.info(f'Guider file {guidefile}')

#----
#- Read and assemble

# fa = Table.read(fafile, 'FIBERASSIGN')
# fa.sort('LOCATION')
with fits.open(fafile) as hdulist:
    fa = hdulist['FIBERASSIGN'].copy()
fa.data.sort(order='LOCATION')


INFO:<ipython-input-9-faa884c9d2c9>:12:<module>: Night 20210922 spectro expid 00101293
INFO:<ipython-input-9-faa884c9d2c9>:13:<module>: Raw data file /Users/weaver/Documents/Data/desi/spectro/data/20210922/00101293/desi-00101293.fits.fz
INFO:<ipython-input-9-faa884c9d2c9>:14:<module>: Fiberassign file /Users/weaver/Documents/Data/desi/target/fiberassign/tiles/trunk/025/fiberassign-025535.fits.gz
INFO:<ipython-input-9-faa884c9d2c9>:16:<module>: Original raw fiberassign file /Users/weaver/Documents/Data/desi/spectro/data/20210922/00101293/fiberassign-025535.fits.gz
INFO:<ipython-input-9-faa884c9d2c9>:17:<module>: Platemaker coordinates file /Users/weaver/Documents/Data/desi/spectro/data/20210922/00101293/coordinates-00101293.fits
INFO:<ipython-input-9-faa884c9d2c9>:18:<module>: Guider file /Users/weaver/Documents/Data/desi/spectro/data/20210922/00101293/guide-00101293.fits.fz


In [10]:
#- if using svn fiberassign override, check consistency of columns that
#- ICS / platemaker used for actual observations; they should never change
if fafile != rawfafile:
    with fits.open(rawfafile) as hdulist:
        rawfa = hdulist['FIBERASSIGN'].copy()
    rawfa.data.sort(order='LOCATION')
    # rawfa = Table.read(rawfafile, 'FIBERASSIGN')
    # rawfa.sort('LOCATION')
    badcol = compare_fiberassign(rawfa.data, fa.data)

    #- special case for tile 80713 on 20210110 with PMRA,PMDEC NaN -> 0.0
    if night == 20210110 and tileid == 80713:
        for col in ['PMRA', 'PMDEC']:
            if col in badcol:
                ii = rawfa[col] != fa[col]
                if np.all(np.isnan(rawfa.data[col][ii]) & (fa.data[col][ii] == 0.0)):
                    log.warning(f'Ignoring {col} mismatch NaN -> 0.0 on tile {tileid} night {night}')
                    badcol.remove(col)

    if len(badcol)>0:
        msg = f'incompatible raw/svn fiberassign files for columns {badcol}'
        log.critical(msg)
        raise ValueError(msg)
    else:
        log.info('svn fiberassign columns used for obervations match raw data (good)')


INFO:<ipython-input-10-4bce83f6698b>:25:<module>: svn fiberassign columns used for obervations match raw data (good)


In [11]:
#- Tiles designed before Fall 2021 had a LOCATION:FIBER swap for fibers
#- 3402 and 3429 at locations 6098 and 6099; check and correct if needed.
#- see desispec #1380
#- NOTE: this only swaps them if the incorrect combination is found
if (6098 in fa.data['LOCATION']) and (6099 in fa.data['LOCATION']):
    iloc6098 = np.where(fa.data['LOCATION'] == 6098)[0][0]
    iloc6099 = np.where(fa.data['LOCATION'] == 6099)[0][0]
    if (fa.data['FIBER'][iloc6098] == 3402) and (fa.data['FIBER'][iloc6099] == 3429):
        log.warning(f'FIBERS 3402 and 3429 are swapped in {fafile}; correcting')
        fa.data['FIBER'][iloc6098] = 3429
        fa.data['FIBER'][iloc6099] = 3402

#- add missing columns for data model consistency
if 'PLATE_RA' not in fa.data.columns.names:
    plate_ra_col = fa.data.columns['TARGET_RA'].copy()
    plate_ra_col.name = 'PLATE_RA'
    fa.data.columns.add_col(plate_ra_col)

if 'PLATE_DEC' not in fa.data.columns.names:
    plate_dec_col = fa.data.columns['TARGET_DEC'].copy()
    plate_dec_col.name = 'PLATE_DEC'
    fa.data.columns.add_col(plate_dec_col)

#- also read extra keywords from HDU 0
fa_hdr0 = fits.getheader(fafile, 0)
if 'OUTDIR' in fa_hdr0:
    fa_hdr0.rename_keyword('OUTDIR', 'FAOUTDIR')
longstrn = fits.Card('LONGSTRN', 'OGIP 1.0', 'The OGIP Long String Convention may be used.')
fa_hdr0.insert('DEPNAM00', longstrn)
fa.header.extend(fa_hdr0, unique=True)
# skipkeys = ['SIMPLE', 'EXTEND', 'COMMENT', 'EXTNAME', 'BITPIX', 'NAXIS']
# addkeys(fa.meta, fa_hdr0, skipkeys=skipkeys)


In [ ]:
fa.header

In [ ]:
rawheader

In [12]:
#- Read platemaker (pm) coordinates file; 3 formats to support:
#  1. has FLAGS_CNT/EXP_n and DX_n, DX_n (e.g. 20201214/00067678)
#  2. has FLAGS_CNT/EXP_n but not DX_n, DY_n (e.g. 20210402/00083144)
#  3. doesn't have any of these (e.g. 20201220/00069029)
# Notes:
#  * don't use FIBER_DX/DY because some files are missing those
#    (e.g. 20210224/00077902)
#  * don't use FLAGS_COR_n because some files are missing that
#    (e.g. 20210402/00083144)

pm = None
numiter = 0
if coordfile is None:
    log.error('No coordinates file, thus no info on fiber positioning')
else:
    with fits.open(coordfile) as hdulist:
        pm = hdulist['DATA'].copy()
    # pm = Table.read(coordfile, 'DATA')  #- PM = PlateMaker

    #- If missing columns *and* not the first in a (split) sequence,
    #- try again with the first expid in the sequence
    #- (e.g. 202010404/00083419 -> 83418)
    if 'DX_0' not in pm.data.columns.names:
        log.error(f'Missing DX_0 in {coordfile}')
        if 'VISITIDS' in rawheader:
            firstexp = int(rawheader['VISITIDS'].split(',')[0])
            if firstexp != rawheader['EXPID']:
                origcorrdfile = coordfile
                coordfile = findfile('coordinates', night, firstexp)
                log.info(f'trying again with {coordfile}')
                with fits.open(coordfile) as hdulist:
                    pm = hdulist['DATA'].copy()
                # pm = Table.read(coordfile, 'DATA')
            else:
                log.error(f'no earlier coordinates file for this tile')
        else:
            log.error('Missing VISITIDS header keywords to find earlier coordinates file')

    if 'FLAGS_CNT_0' not in pm.data.columns.names:
        log.error(f'Missing spotmatch FLAGS_CNT_0 in {coordfile}; no positioner offset info')
        pm = None
        numiter = 0
    else:
        #- Count number of iterations in file
        numiter = len([col for col in pm.data.columns.names if col.startswith('FLAGS_CNT_')])
        log.info(f'Using FLAGS_CNT_{numiter-1} in {coordfile}')


ERROR:<ipython-input-12-35809e5c20af>:24:<module>: Missing DX_0 in /Users/weaver/Documents/Data/desi/spectro/data/20210922/00101293/coordinates-00101293.fits
DEBUG:meta.py:170:findfile: hpixdir = 'hpix'
DEBUG:meta.py:181:findfile: rawdata_dir = '/Users/weaver/Documents/Data/desi/spectro/data'
INFO:<ipython-input-12-35809e5c20af>:30:<module>: trying again with /Users/weaver/Documents/Data/desi/spectro/data/20210922/00101292/coordinates-00101292.fits
INFO:<ipython-input-12-35809e5c20af>:46:<module>: Using FLAGS_CNT_1 in /Users/weaver/Documents/Data/desi/spectro/data/20210922/00101292/coordinates-00101292.fits


In [ ]:
fa

In [13]:
#- Now let's merge that platemaker coordinates table (pm) with fiberassign
if pm is not None:
    pm_table = Table(pm.data)
    pm_table['LOCATION'] = 1000*pm_table['PETAL_LOC'] + pm_table['DEVICE_LOC']
    keep = np.in1d(pm_table['LOCATION'], fa.data['LOCATION'])
    pm_table = pm_table[keep]
    pm_table.sort('LOCATION')
    log.info('{}/{} fibers in coordinates file'.format(len(pm_table), len(fa.data)))

    #- Create fibermap table to merge with fiberassign file
    fibermap = Table()
    fibermap_header = fits.Header({'XTENSION': 'BINTABLE'})
    fibermap['LOCATION'] = pm_table['LOCATION']
    fibermap['NUM_ITER'] = numiter

    #- Sometimes these columns are missing in the coordinates files, maybe
    #- only when numiter=1, i.e. only a blind move but not corrections?
    if f'FPA_X_{numiter-1}' in pm_table.colnames:
        fibermap['FIBER_X'] = pm_table[f'FPA_X_{numiter-1}']
        fibermap['FIBER_Y'] = pm_table[f'FPA_Y_{numiter-1}']
        fibermap['DELTA_X'] = pm_table[f'DX_{numiter-1}']
        fibermap['DELTA_Y'] = pm_table[f'DY_{numiter-1}']
    else:
        log.error('No FIBER_X/Y or DELTA_X/Y information from platemaker')
        fibermap['FIBER_X'] = np.zeros(len(pm_table))
        fibermap['FIBER_Y'] = np.zeros(len(pm_table))
        fibermap['DELTA_X'] = np.zeros(len(pm_table))
        fibermap['DELTA_Y'] = np.zeros(len(pm_table))

    if ('FIBER_RA' in pm_table.colnames) and ('FIBER_DEC' in pm_table.colnames):
        fibermap['FIBER_RA'] = pm_table['FIBER_RA']
        fibermap['FIBER_DEC'] = pm_table['FIBER_DEC']
    else:
        log.error('No FIBER_RA or FIBER_DEC from platemaker')
        fibermap['FIBER_RA'] = np.zeros(len(pm_table))
        fibermap['FIBER_DEC'] = np.zeros(len(pm_table))

    #- Bit definitions at https://desi.lbl.gov/trac/wiki/FPS/PositionerFlags

    #- FLAGS_EXP bit 2 is for positioners (not FIF, GIF, ...)
    #- These should match what is in fiberassign, except broken fibers
    expflags = pm_table[f'FLAGS_EXP_{numiter-1}']
    goodmatch = ((expflags & 4) == 4)
    if np.any(~goodmatch):
        badloc = list(pm_table['LOCATION'][~goodmatch])
        log.warning(f'Flagging {len(badloc)} locations without POS_POS bit set: {badloc}')

    #- Keep only matched positioners (FLAGS_CNT_n bit 0)
    cntflags = pm_table[f'FLAGS_CNT_{numiter-1}']
    spotmatched = ((cntflags & 1) == 1)

    num_nomatch = np.sum(goodmatch & ~spotmatched)
    if num_nomatch > 0:
        badloc = list(pm_table['LOCATION'][goodmatch & ~spotmatched])
        log.error(f'Flagging {num_nomatch} unmatched fiber locations: {badloc}')

    goodmatch &= spotmatched

    #- pass forward dummy column for joining with fiberassign
    fibermap['_GOODMATCH'] = goodmatch

    #- WARNING: this join can re-order the table
    fibermap = join(Table(fa.data), fibermap, join_type='left')

    #- poor and bad positioning
    dr = np.sqrt(fibermap['DELTA_X']**2 + fibermap['DELTA_Y']**2) * 1000
    poorpos = ((poor_offset_um < dr) & (dr <= bad_offset_um))
    badpos = (dr > bad_offset_um) | np.isnan(dr)
    numpoor = np.count_nonzero(poorpos)
    numbad = np.count_nonzero(badpos)
    if numpoor > 0:
        log.warning(f'Flagging {numpoor} POOR positions with {poor_offset_um} < offset <= {bad_offset_um} microns')
    if numbad > 0:
        log.warning(f'Flagging {numbad} BAD positions with offset > {bad_offset_um} microns')

    #- Set fiber status bits
    missing = np.in1d(fibermap['LOCATION'], pm_table['LOCATION'], invert=True)
    missing |= ~fibermap['_GOODMATCH']
    fibermap['FIBERSTATUS'][missing] |= fibermask.MISSINGPOSITION
    fibermap['FIBERSTATUS'][poorpos] |= fibermask.POORPOSITION
    fibermap['FIBERSTATUS'][badpos] |= fibermask.BADPOSITION

    fibermap.remove_column('_GOODMATCH')

else:
    #- No coordinates file or no positioning iterations;
    #- just use fiberassign + dummy columns
    log.error('Unable to find useful coordinates file; proceeding with fiberassign + dummy columns')
    fibermap = Table(fa.data)
    fibermap['NUM_ITER'] = 0
    fibermap['FIBER_X'] = 0.0
    fibermap['FIBER_Y'] = 0.0
    fibermap['DELTA_X'] = 0.0
    fibermap['DELTA_Y'] = 0.0
    fibermap['FIBER_RA'] = 0.0
    fibermap['FIBER_DEC'] = 0.0
    # Update data types to be consistent with updated value if coord file was used.
    for val in ['FIBER_X','FIBER_Y','DELTA_X','DELTA_Y']:
        old_col = fibermap[val]
        fibermap.replace_column(val,Table.Column(name=val,data=old_col.data,dtype='>f8'))
    for val	in ['LOCATION','NUM_ITER']:
        old_col = fibermap[val]
        fibermap.replace_column(val,Table.Column(name=val,data=old_col.data,dtype=np.int64))


INFO:<ipython-input-13-ae6edef95a0b>:8:<module>: 5000/5000 fibers in coordinates file


In [14]:
#- Update SKY and STD target bits to be in both CMX_TARGET and DESI_TARGET
#- i.e. if they are set in one, also set in the other.  Ditto for SV*
for targetcol in ['CMX_TARGET', 'SV0_TARGET', 'SV1_TARGET', 'SV2_TARGET']:
    if targetcol in fibermap.colnames:
        for mask in [
                desi_mask.SKY, desi_mask.STD_FAINT, desi_mask.STD_BRIGHT]:
            ii  = (fibermap[targetcol] & mask) != 0
            iidesi = (fibermap['DESI_TARGET'] & mask) != 0
            fibermap[targetcol][iidesi] |= mask
            fibermap['DESI_TARGET'][ii] |= mask


In [15]:
#- Add header information from rawfile
log.debug(f'Adding header keywords from {rawfile}')
# skipkeys = ['EXPTIME',]
# addkeys(fibermap.meta, rawheader, skipkeys=skipkeys)
fibermap_header.extend(rawheader, strip=True)
fibermap_header.remove('EXPTIME')
fibermap['EXPTIME'] = rawheader['EXPTIME']
#- Add header info from guide file
#- sometimes full header is in HDU 0, other times HDU 1...
if guidefile is not None:
    log.debug(f'Adding header keywords from {guidefile}')
    guideheader = fits.getheader(guidefile, 0)
    if 'TILEID' not in guideheader:
        guideheader = fits.getheader(guidefile, 1)

    if fibermap_header['TILEID'] != guideheader['TILEID']:
        raise RuntimeError('fiberassign tile {} != guider tile {}'.format(
            fibermap_header['TILEID'], guideheader['TILEID']))

    # addkeys(fibermap.meta, guideheader, skipkeys=skipkeys)
    fibermap_header.extend(guideheader, strip=True)
    fibermap_header.remove('EXPTIME')


fibermap_header['EXTNAME'] = 'FIBERMAP'
for key in ('ZIMAGE', 'ZSIMPLE', 'ZBITPIX', 'ZNAXIS', 'ZNAXIS1', 'ZTILE1', 'ZCMPTYPE', 'ZNAME1', 'ZVAL1', 'ZNAME2', 'ZVAL2'):
    fibermap_header.remove(key)

DEBUG:<ipython-input-15-e3052427c145>:2:<module>: Adding header keywords from /Users/weaver/Documents/Data/desi/spectro/data/20210922/00101293/desi-00101293.fits.fz
DEBUG:<ipython-input-15-e3052427c145>:11:<module>: Adding header keywords from /Users/weaver/Documents/Data/desi/spectro/data/20210922/00101293/guide-00101293.fits.fz


In [16]:
fibermap_header

XTENSION= 'BINTABLE'                                                            
EXTNAME = 'FIBERMAP'                                                            
MODULE  = 'CI      '           / Image Sources/Component                        
EXPID   =               101293 / Exposure number                                
EXPFRAME=                    0 / Frame number                                   
FRAMES  =                      / Number of Frames in Archive                    
COSMSPLT=                    T / Cosmics split exposure if true                 
MAXSPLIT=                    0 / Number of allowed exposure splits              
VISITIDS= '101292,101293'      / List of expids for a visit (same tile)         
TILEID  =                25535 / DESI Tile ID                                   
FIBASSGN= '/data/tiles/SVN_tiles/025/fiberassign-025535.fits.gz' / Fiber assign 
FLAVOR  = 'science '           / Observation type                               
OBSTYPE = 'SCIENCE '        

In [17]:
#- Early data raw headers had bad >8 char 'FIBERASSIGN' keyword
if 'FIBERASSIGN' in fibermap_header:
    log.warning('Renaming header keyword FIBERASSIGN -> FIBASSGN')
    fibermap_header.rename_keyword('FIBERASSIGN', 'FIBASSGN')
    # fibermap.meta['FIBASSGN'] = fibermap.meta['FIBERASSIGN']
    # del fibermap.meta['FIBERASSIGN']

#- similarly for early splits in raw data file
if 'USESPLITS' in fibermap_header:
    log.warning('Renaming header keyword USESPLITS -> USESPLIT')
    fibermap_header.rename_keyword('USESPLITS', 'USESPLIT')
    # fibermap.meta['USESPLIT'] = fibermap.meta['USESPLITS']
    # del fibermap.meta['USESPLITS']

#- Record input guide and coordinates files
if guidefile is not None:
    fibermap_header['GUIDEFIL'] = os.path.basename(guidefile)
else:
    fibermap_header['GUIDEFIL'] = 'MISSING'

if coordfile is not None:
    fibermap_header['COORDFIL'] = os.path.basename(coordfile)
else:
    fibermap_header['COORDFIL'] = 'MISSING'


In [18]:
badamps = None
#- mask the fibers defined by badamps
if badamps is not None:
    maskbits = {'b':fibermask.BADAMPB, 'r':fibermask.BADAMPR, 'z':fibermask.BADAMPZ}
    ampoffsets = {'A': 0, 'B':250, 'C':0, 'D':250}
    for (camera, petal, amplifier) in parse_badamps(badamps):
        maskbit = maskbits[camera]
        ampoffset = ampoffsets[amplifier]
        fibermin = int(petal)*500 + ampoffset
        fibermax = fibermin + 250
        ampfibs = np.arange(fibermin,fibermax)
        truefmax = fibermax - 1
        log.info(f'Masking fibers from {fibermin} to {truefmax} for camera {camera} because of badamp entry '+\
                 f'{camera}{petal}{amplifier}')
        ampfiblocs = np.in1d(fibermap['FIBER'], ampfibs)
        fibermap['FIBERSTATUS'][ampfiblocs] |= maskbit


In [19]:
badfibers_filename = None
#- mask the fibers defined by bad fibers
if badfibers_filename is not None:
    table=Table.read(badfibers_filename)

    # list of bad fibers that are in the fibermap
    badfibers=np.intersect1d(np.unique(table["FIBER"]),fibermap["FIBER"])

    for i,fiber in enumerate(badfibers) :
        # for each of the bad fiber, add the bad bits to the fiber status
        badfibermask = np.bitwise_or.reduce(table["FIBERSTATUS"][table["FIBER"]==fiber])
        fibermap['FIBERSTATUS'][fibermap["FIBER"]==fiber] |= badfibermask


In [20]:
#- NaN are a pain; reset to dummy values
for col in [
    'FIBER_X', 'FIBER_Y',
    'DELTA_X', 'DELTA_Y',
    'FIBER_RA', 'FIBER_DEC',
    'GAIA_PHOT_G_MEAN_MAG',
    'GAIA_PHOT_BP_MEAN_MAG',
    'GAIA_PHOT_RP_MEAN_MAG',
    ]:
    ii = np.isnan(fibermap[col])
    if np.any(ii):
        n = np.sum(ii)
        log.warning(f'Setting {n} {col} NaN to 0.0')
        fibermap[col][ii] = 0.0
#
# Some SV1-era files had these extraneous columns, make sure they are not propagated.
#
for col in ('NUMTARGET', 'BLOBDIST', 'FIBERFLUX_IVAR_G', 'FIBERFLUX_IVAR_R', 'FIBERFLUX_IVAR_Z', 'HPXPIXEL'):
    if col in fibermap.colnames:
        log.debug("Removing column '%s' from fibermap table.", col)
        fibermap.remove_column(col)

#
# Some SV1-era files have RELEASE as int32.  Should be int16.
#
if fibermap['RELEASE'].dtype == np.dtype('>i2'):
    log.debug("RELEASE has correct type.")
else:
    log.warning("Setting RELEASE to int16.")
    fibermap['RELEASE'] = fibermap['RELEASE'].astype(np.int16)
    
#- Some code incorrectly relies upon the fibermap being sorted by
#- fiber number, so accomodate that before returning the table
fibermap.sort('FIBER')


DEBUG:<ipython-input-20-027063a3dbd1>:27:<module>: RELEASE has correct type.


In [22]:
fibermap

TARGETID,PETAL_LOC,DEVICE_LOC,LOCATION,FIBER,FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,REF_EPOCH,LAMBDA_REF,FA_TARGET,FA_TYPE,OBJTYPE,FIBERASSIGN_X,FIBERASSIGN_Y,PRIORITY,SUBPRIORITY,OBSCONDITIONS,RELEASE,BRICKNAME,BRICKID,BRICK_OBJID,MORPHTYPE,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,MASKBITS,SERSIC,SHAPE_R,SHAPE_E1,SHAPE_E2,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_RP_MEAN_MAG,PARALLAX,PHOTSYS,PRIORITY_INIT,NUMOBS_INIT,DESI_TARGET,BGS_TARGET,MWS_TARGET,SCND_TARGET,PLATE_RA,PLATE_DEC,NUM_ITER,FIBER_X,FIBER_Y,DELTA_X,DELTA_Y,FIBER_RA,FIBER_DEC,EXPTIME
int64,int16,int32,int64,int32,int32,float64,float64,float32,float32,float32,float32,int64,uint8,str3,float32,float32,int32,float64,int32,int16,str8,int32,int32,str4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,int64,str2,float32,float32,float32,float32,str1,int64,int64,int64,int64,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64
39627820267212263,0,311,311,0,0,318.33200642408787,1.1505490614355565,0.0,0.0,2015.5,5400.0,1152921504606846976,1,TGT,86.89384,-286.61023,2100,0.7591475862904349,516,9010,3188p010,338841,3559,SER,0.14201052,2.922428,11.564456,25.054604,29.246515,18.437819,1517.8446,734.36426,121.89684,2.3242793,0.60445595,1.3620845,5.389959,11.677443,1.3625942,5.391116,11.679006,0,6.0,0.7316475,-0.091552466,0.2594839,0,,0.0,0.0,0.0,0.0,S,2100,2,1152921504606846976,65545,0,0,318.33200642408787,1.1505490614355565,2,86.916,-286.624,-0.008,-0.0,318.3319743849711,1.1505494954124924,32.1984
39627820267216380,0,272,272,1,0,318.4386646327084,1.1921055836897205,0.0,0.0,2015.5,5400.0,1152921504606846976,1,TGT,60.21497,-275.50952,2000,0.6808023096715565,516,9010,3188p017,338841,7676,PSF,0.15468709,1.7510165,9.293649,24.494495,45.027287,29.354399,1438.9269,603.1336,87.19041,2.0541596,0.5560471,0.70157367,3.7236538,9.814123,0.70157367,3.7236538,9.814123,0,4.0,0.9347005,0.14849046,-0.2538432,0,G2,0.0,0.0,0.0,0.0,S,2000,2,1152921504606846976,65537,0,0,318.4386646327084,1.1921055836897205,2,60.229,-275.521,-0.004,-0.001,318.4386485123608,1.1921096151996238,32.1984
39627820271405576,0,252,252,2,0,318.56939535092124,1.209281012483101,0.26090968,2.2330418,2015.5,5400.0,2305843009213693952,1,TGT,27.74554,-270.8191,1500,0.8463527955371811,516,9010,3188p012,338842,2568,PSF,0.15451324,53.313652,100.33186,137.51811,47.86699,25.6404,467.71194,266.9358,130.3859,2.323597,0.6722828,41.483948,78.06934,107.00439,41.483948,78.06934,107.00439,0,0.0,0.0,0.0,0.0,2690537285682587264,G2,17.609327,18.110582,16.994308,0.4586911,S,1500,2,2305843009213693952,0,1280,0,318.56939535092124,1.209281012483101,2,27.754,-270.823,-0.003,-0.006,318.5693831897469,1.2093044098815495,32.1984
39627826311207678,0,156,156,3,0,318.7006640401452,1.4116735600742132,-0.0867894,-1.2922232,2015.5,5400.0,2305843009213693952,1,,-4.673432,-219.10924,1500,0.8656495549602405,516,9010,,340282,6910,,0.116076544,35.83482,39.535065,39.101536,6.774912,3.2722197,714.10583,595.2669,249.3519,3.0328572,0.7072545,27.909555,30.791449,30.4538,27.909555,30.791449,30.4538,0,0.0,0.0,0.0,0.0,2690593807452445824,,18.495955,18.588202,18.13337,0.5264187,,1500,2,2305843009213693952,0,1280,0,318.7006640401452,1.4116735600742132,2,-4.672,-219.109,-0.003,-0.003,318.70065187629086,1.4116854708359312,32.1984
39627820275598098,0,198,198,4,0,318.7688058093043,1.2722292045806904,0.0,0.0,2015.5,5400.0,1152921504606846976,1,SKY,-21.58714,-254.55681,2100,0.4482942980243694,516,9010,3186p010,338843,786,,0.11373752,16.617014,33.145374,57.3303,59.2111,45.53512,551.9292,272.6623,43.093155,1.7897556,0.48807216,2.6933048,5.372241,9.292161,2.6942801,5.3728986,9.292161,0,1.0525084,2.36795,0.058085084,0.42429152,0,,0.0,0.0,0.0,0.0,,2100,2,115

In [23]:
fibermap_header

XTENSION= 'BINTABLE'                                                            
EXTNAME = 'FIBERMAP'                                                            
MODULE  = 'CI      '           / Image Sources/Component                        
EXPID   =               101293 / Exposure number                                
EXPFRAME=                    0 / Frame number                                   
FRAMES  =                      / Number of Frames in Archive                    
COSMSPLT=                    T / Cosmics split exposure if true                 
MAXSPLIT=                    0 / Number of allowed exposure splits              
VISITIDS= '101292,101293'      / List of expids for a visit (same tile)         
TILEID  =                25535 / DESI Tile ID                                   
FIBASSGN= '/data/tiles/SVN_tiles/025/fiberassign-025535.fits.gz' / Fiber assign 
FLAVOR  = 'science '           / Observation type                               
OBSTYPE = 'SCIENCE '        

In [24]:
fibermap_hdu = fits.BinTableHDU(fibermap)
fibermap_hdu.header.extend(fibermap_header, unique=True)

In [25]:
fibermap_hdulist = fits.HDUList([fits.PrimaryHDU(), fibermap_hdu])

In [26]:
fibermap_hdulist.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  FIBERMAP      1 BinTableHDU    817   5000R x 70C   ['K', 'I', 'J', 'K', 'J', 'J', 'D', 'D', 'E', 'E', 'E', 'E', 'K', 'B', '3A', 'E', 'E', 'J', 'D', 'J', 'I', '8A', 'J', 'J', '4A', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'I', 'E', 'E', 'E', 'E', 'K', '2A', 'E', 'E', 'E', 'E', '1A', 'K', 'K', 'K', 'K', 'K', 'K', 'D', 'D', 'K', 'D', 'D', 'D', 'D', 'D', 'D', 'D']   


In [27]:
fibermap_hdulist[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  

In [30]:
foo = np.array([1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000], dtype=np.int32)

In [31]:
foo.astype(np.int16)

array([ 1000,  2000,  3000,  4000,  5000,  6000,  7000,  8000,  9000,
       10000], dtype=int16)

## Look at the list of exposures.

In [4]:
exposures = os.path.join(os.environ['DESI_SPECTRO_REDUX'], os.environ['SPECPROD'], "exposures-{SPECPROD}.fits".format(**os.environ))
with fits.open(exposures) as hdulist:
    exposures = hdulist['EXPOSURES'].data.copy()

In [12]:
program = faflavor2program(exposures['FAFLAVOR'])
sv1bright = (program == 'bright') & (exposures['SURVEY'] == 'sv1') & (exposures['NIGHT'] == 20210224)
exposures['NIGHT'][sv1bright], exposures['EXPID'][sv1bright]

(array([20210224, 20210224, 20210224, 20210224, 20210224, 20210224,
        20210224, 20210224, 20210224], dtype=int32),
 array([77902, 77903, 77924, 77926, 77929, 77930, 77931, 77951, 77952],
       dtype=int32))

In [15]:
exposures_row = np.nonzero((exposures['NIGHT'] == 20210224) & (exposures['EXPID'] == 77952))[0][0]
survey_program = "{0}-{1}".format(exposures['SURVEY'][exposures_row], program[exposures_row])
survey_program

'sv1-bright'

In [7]:
sv1dark = (program == 'dark') & (exposures['SURVEY'] == 'sv1') & (exposures['NIGHT'] == 20210220)
exposures['NIGHT'][sv1dark], exposures['EXPID'][sv1dark]

(array([20210220, 20210220, 20210220, 20210220, 20210220, 20210220],
       dtype=int32),
 array([77254, 77256, 77257, 77258, 77259, 77260], dtype=int32))

In [12]:
sv2dark = (program == 'dark') & (exposures['SURVEY'] == 'sv2') & (exposures['NIGHT'] == 20210403) 
exposures['NIGHT'][sv2dark], exposures['EXPID'][sv2dark]

(array([20210403, 20210403, 20210403, 20210403, 20210403, 20210403],
       dtype=int32),
 array([83267, 83268, 83269, 83270, 83271, 83272], dtype=int32))

In [15]:
sv2bright = (program == 'bright') & (exposures['SURVEY'] == 'sv2') & (exposures['NIGHT'] == 20210324) 
exposures['NIGHT'][sv2bright], exposures['EXPID'][sv2bright]

(array([20210324, 20210324, 20210324, 20210324, 20210324, 20210324,
        20210324, 20210324, 20210324, 20210324, 20210324, 20210324,
        20210324, 20210324, 20210324, 20210324, 20210324], dtype=int32),
 array([81839, 81840, 81841, 81845, 81846, 81847, 81848, 81849, 81850,
        81853, 81854, 81855, 81856, 81857, 81859, 81860, 81861],
       dtype=int32))

In [18]:
sv3dark = (program == 'dark') & (exposures['SURVEY'] == 'sv3') & (exposures['NIGHT'] == 20210420) 
exposures['NIGHT'][sv3dark], exposures['EXPID'][sv3dark]

(array([20210420, 20210420, 20210420, 20210420, 20210420, 20210420,
        20210420, 20210420, 20210420], dtype=int32),
 array([85616, 85629, 85630, 85634, 85639, 85643, 85644, 85645, 85646],
       dtype=int32))

In [20]:
sv3bright = (program == 'bright') & (exposures['SURVEY'] == 'sv3') & (exposures['NIGHT'] == 20210420) 
exposures['NIGHT'][sv3bright], exposures['EXPID'][sv3bright]

(array([20210420, 20210420, 20210420, 20210420, 20210420, 20210420,
        20210420, 20210420, 20210420, 20210420, 20210420, 20210420,
        20210420, 20210420, 20210420, 20210420, 20210420], dtype=int32),
 array([85617, 85620, 85621, 85622, 85623, 85624, 85626, 85627, 85628,
        85631, 85632, 85633, 85635, 85636, 85637, 85640, 85642],
       dtype=int32))

In [22]:
maindark = (program == 'dark') & (exposures['SURVEY'] == 'main')  & (exposures['NIGHT'] == 20210531) 
exposures['NIGHT'][maindark], exposures['EXPID'][maindark]

(array([20210531, 20210531, 20210531, 20210531, 20210531, 20210531,
        20210531, 20210531, 20210531, 20210531, 20210531, 20210531,
        20210531, 20210531, 20210531, 20210531, 20210531, 20210531,
        20210531], dtype=int32),
 array([90508, 90509, 90511, 90512, 90513, 90514, 90515, 90516, 90517,
        90518, 90519, 90520, 90521, 90522, 90523, 90524, 90525, 90526,
        90527], dtype=int32))

In [23]:
mainbright = (program == 'bright') & (exposures['SURVEY'] == 'main')  & (exposures['NIGHT'] == 20210531) 
exposures['NIGHT'][mainbright], exposures['EXPID'][mainbright]

(array([20210531, 20210531, 20210531, 20210531, 20210531, 20210531,
        20210531], dtype=int32),
 array([90503, 90504, 90505, 90506, 90507, 90510, 90528], dtype=int32))

## Dealing with LONGSTRN
Add this header
```
LONGSTRN= 'OGIP 1.0'           / The OGIP Long String Convention may be used.
```
[Reference](https://heasarc.gsfc.nasa.gov/docs/software/fitsio/c/f_user/node25.html).